## **Sparse Matrices and LU Factorization Kira Novitchkova-Burbank**

# Why to use sparse matrix format instead of regular matrix format ?


$\bullet$
Sparse matrix is the one which has most of the elements as zeros as opposed to dense which has most of the elements as non-zeros. Provided with large matrix, it is common that most of the elements are zeros. Therefore, it makes sense to use only non-zero values to perform operations as zero times anything will always give zero.

$\bullet$
It is not uncommon that some spare matrix only have less than 1% density. If we save them in dense matrix form, the memory usage will be much much higher


$\bullet$
Here we create an Identity Matrix in Coo format ,then to compare the memory useage, we create the same Identity matrix in the dense form with ```Numpy```

In [ ]:
from sys import getsizeof
import numpy as np

row = np.arange(0,1000)
col = np.arange(0,1000)
data = np.ones(1000)
print(f'memory usage for  coo format is {getsizeof(data)*3} bytes')

dense_matrix = np.eye(1000)
print(f'memory usage for dense format is {getsizeof(dense_matrix)} bytes')

print(f'The memory usage of the sparse format is dense formats {((getsizeof(data)*3)/getsizeof(dense_matrix))*100} %')

memory usage for  coo format is 24336 bytes
memory usage for dense format is 8000128 bytes
The memory usage of the sparse format is dense formats 0.304195132877874 %


Cool visualization for some sparse matrix:

![](https://storage.googleapis.com/lds-media/images/sparse-matrix-visualization.width-1200.jpg)

## Pacakge for spare matrix

The most popular package for sparse matrix in python is ```Scipy.sparse```

[Link Text](https://docs.scipy.org/doc/scipy/reference/sparse.html)

## Constructing a coo matrix using i,j,v format

In [ ]:
from scipy.sparse import coo_matrix

row  = np.array([0, 3, 1, 0])
col  = np.array([0, 3, 1, 2])
data = np.array([4, 5, 7, 9])
coo_array = coo_matrix((data, (row, col)), shape=(4, 4))
print(coo_array.toarray())

[[4 0 9 0]
 [0 7 0 0]
 [0 0 0 0]
 [0 0 0 5]]


## Constructing a csr matrix using i,j,v format

In [ ]:
from scipy.sparse import csr_matrix

indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
csr_array = csr_matrix((data, indices, indptr), shape=(3, 3))
print(csr_array.toarray())

[[1 0 2]
 [0 0 3]
 [4 5 6]]


# convert from COO to CSR or vice versa

In [ ]:
converted_to_csr = coo_array.tocsr()

print(f'csr data is {converted_to_csr.data}')
print(f'csr indptr is {converted_to_csr.indptr}')
print(f'csr indices is {converted_to_csr.indices}')

converted_to_coo = csr_array.tocoo()

print(f'coo row is {converted_to_coo.row}')
print(f'coo col is {converted_to_coo.col}')
print(f'coo data is {converted_to_coo.data}')

csr data is [4 9 7 5]
csr indptr is [0 2 3 3 4]
csr indices is [0 2 1 3]
coo row is [0 0 1 2 2 2]
coo col is [0 2 2 0 1 2]
coo data is [1 2 3 4 5 6]


# Reading the data

Here is a function written for you to read the ```.txt``` or ```.csv``` file. if you prefer other ways, please feel free to use your own function to read the matrix file if you like.

In [ ]:
import pandas

def read_data(filepath):
    data = pandas.read_csv(filepath,
                         sep=" ",
                         header=None)
    data.columns = ["col", "row", "data"]
    matrix = coo_matrix((data["data"], (data["col"], data["row"])))

    return matrix

In [ ]:
matrix_25 = read_data('25-1.txt')

## LU factorization with existing package

feel free to write your own LU factorization package if you like.

$\bullet$
Note: the return matrix is in default coo format, to use in csr format you need to run ```.tocsr()``` to get it's ```data```,```indptr``` etc

In [ ]:
from scipy.sparse.linalg import splu

def LU_factor(sparse_matrix):

  lu = splu(matrix_25.tocsc(),
          permc_spec="NATURAL",
          diag_pivot_thresh=0,
          options={"SymmetricMode":True})
  return lu.L , lu.U


In [ ]:
lower_matrix,upper_matrix = LU_factor(matrix_25)

#Ly=b
converted_to_csr_lower = lower_matrix.tocsr()

print(f'csr data is {converted_to_csr_lower.data}')
print(f'csr indptr is {converted_to_csr_lower.indptr}')
print(f'csr indices is {converted_to_csr_lower.indices}')

#Ux=y

converted_to_csr_upper = upper_matrix.tocsr()

print(f'csr data is {converted_to_csr_upper.data}')
print(f'csr indptr is {converted_to_csr_upper.indptr}')
print(f'csr indices is {converted_to_csr_upper.indices}')

csr data is [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  7.81651337e-03
  1.00000000e+00  1.00000000e+00  3.42081422e-02  1.00000000e+00
  4.79411079e-03  1.52951796e-01  1.00000000e+00  2.24172763e-01
  2.11306937e-01  1.00000000e+00 -5.61051613e-01  5.55710429e-02
  1.00000000e+00  1.00000000e+00 -9.13253384e-03  9.56259860e-03
  5.36202692e-03  1.00000000e+00  5.54053284e-01 -8.54958562e-02
  9.76828509e-01  7.97925165e-02  1.00000000e+00  9.55534800e-02
 -9.46438157e-03  2.44283726e-01 -1.67316578e-02  1.00000000e+00
 -1.73724916e-02  1.81905884e-02  1.01999915e-02 -2.77126218e-01
  1.00000000e+00  1.09871734e-01 -1.08825761e-02  2.80888531e-01
 -1.92388206e-02 -5.14205968e-01  1.00000000e+00  1.12717661e-01
  7.11021243e-02 -3.79564544e-03 -4.75689401e-02 -1.56905275e-17
 -4.41435684e-17  1.00000000e+00 -1.09586899e+00  2.05363146e-01
 -1.09629030e-02 -1.37392621e-01 -5.58582778e-16 -1.57310255e-15
  2.53491788e-01  1.00000000e+00  3.41763988e-03 -4.03378532e-01
 -2.26186065e

# Backward elimation with regular matrix format
Here is a simple example of how to write backward elimation with dense formate matrix.

In [ ]:
np.random.seed(42)
n = 5
A = np.random.randn(n, n) * np.tri(n).T #making a upper tri matrix with 5x5 size
x = np.ones(5)

b = A @ x

print(A)
print('\n')
print(f'Our target vector is {b}')
print('\n')

x_comparision = np.zeros(n) #create a all zero vector to write in

#backward subsitution
for i in range(n-1, -1, -1):
    tmp = b[i]
    for j in range(n-1, i, -1):
        tmp -= x_comparision[j]*A[i,j]
    x_comparision[i] = tmp/A[i,i]


print('True solution:',x)
print('Back sub solution:',x_comparision)
print('Difference:',np.linalg.norm(x-x_comparision))

[[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337]
 [-0.          1.57921282  0.76743473 -0.46947439  0.54256004]
 [-0.         -0.          0.24196227 -1.91328024 -1.72491783]
 [-0.         -0.          0.         -0.90802408 -1.4123037 ]
 [ 0.         -0.          0.         -0.         -0.54438272]]


Our target vector is [ 2.29501487  2.4197332  -3.39623581 -2.32032778 -0.54438272]


True solution: [1. 1. 1. 1. 1.]
Back sub solution: [1. 1. 1. 1. 1.]
Difference: 4.173316387848236e-15


# Your Code here:

be careful how to handle ```indptr```,```indices```. try follow the basic structure of the regular backward substitution and change accordingly.

In [ ]:
### your code here ###

#Note: Some of my code is in the "LU factorization with existing package" section
#      It's the code for converted_to_csr_lower and converted_to_csr_upper



#Ly=b
def forward_subs(lower, b):

  rowptr_lower = converted_to_csr_lower.indptr
  data_lower = converted_to_csr_lower.data
  J_lower = converted_to_csr_lower.indices

  n = matrix_25.shape[0]
  x = np.ones(n)
  y = np.zeros(n)

  for i in range(0, n):

    y[i] = b[i]

    for k in range(rowptr_lower[i], rowptr_lower[i+1]):
      j = J_lower[k]
      print(k)

      if (j < i):
        y[i] -= data_lower[k] * y[j]

      if (j == i):
        diag = data_lower[k]

    y[i] /= diag

  return y



#Ux=y
def backward_sub(upper, y):


  rowptr_upper = converted_to_csr_upper.indptr
  data_upper = converted_to_csr_upper.data
  J_upper = converted_to_csr_upper.indices

  n = matrix_25.shape[0]
  x = np.zeros(n)

  for i in range(n-1, -1, -1):

    x[i] = y[i]

    for k in range(rowptr_upper[i+1], rowptr_upper[i], -1):
      j = J_upper[k-1]

      if (j > i):
        x[i] -= data_upper[k-1] * x[j]

      if (j == i):
        diag = data_upper[k-1]

    x[i] /= diag

  return x


#main
b = np.ones(matrix_25.shape[0])

y = forward_subs(converted_to_csr_lower, b)
print("y:", y)

x = backward_sub(converted_to_csr_upper, y)
print("x:", x)

#######################

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
y: [ 1.          1.          1.          0.99218349  1.          0.96579186
  0.84344964  0.5645203   1.52968063  1.          0.99512208 -0.14205402
  0.53373647  1.26649533  0.73831948  0.84619251  1.76268704  1.05705645
  0.14889538  0.83109684  0.87678763  1.73161928  0.64069191  2.08140716
  0.91282287]
x: [ 79.98231453  10.15748493 -12.25701526   9.53035369  14.6850587
   2.49633451   7.06801677   0.29547416   2.05673994   5.62341325
   8.38769248   2.95537503  13.28639585   9.8085526   13.77884731
  15.03131029   6.30920172  16.01739526  25.50740517  31.28434704
  30.89510973  32.52018986  38.56028089   2.11508874  8

# Test your solution

you should be expecting something like this

In [ ]:
matrix_25 = read_data('25-1.txt')

lower_25,upper_25 = LU_factor(matrix_25)

x_vector = np.ones((matrix_25.shape[0],1))  # ground truth is all 1 vector

b = matrix_25 @ x_vector

y = forward_subs(lower_25,b)
x_solution = backward_sub(upper_25,y)

print('difference between output and ground truth is :',np.linalg.norm(x_vector - x_solution))

difference between output and ground truth is : 1.2994827337208943e-14


difference between output and ground truth is : 2.9666023450863638e-15